In [1]:
# QMR execution settings
plot_sne = False                         # t-SNE plot? (requires a bit of RAM)
plot_metrics_comp = False                 # metrics comparison?
savefig = False                           # save figs or show in line
use_existing_metrics = True              # read existing metrics output data files instead of processing them?
regressor_quality_metrics = ['sigma','snr','rer','sharpness','scale','score']

#Define path of the original (reference) datasets
data_paths = {
    "DeepGlobe469": "./Data/DeepGlobe",
    "UCMerced2100": "./Data/UCMerced_LandUse",
    "UCMerced380": "./Data/test-ds",
    "USGS279": "./Data/USGS",
    "shipsnet-ships4000": "./Data/shipsnet",
    "shipsnet-scenes7": "./Data/shipsnet",
    "inria-test10": "./Data/inria-aid_short",
    "inria-test180": "./Data/AerialImageDataset",
    "inria-train180": "./Data/AerialImageDataset",
    "XView-train846": "./Data/XView",
    "XView-val281": "./Data/XView",
}
image_folders = {
    "DeepGlobe469": "images",
    "UCMerced2100": "Images/ALL_CATEGORIES",
    "UCMerced380": "test",
    "USGS279": "hr_images",
    "shipsnet-ships4000": "shipsnet",
    "shipsnet-scenes7": "scenes/scenes",
    "inria-test10": "test/images_short",
    "inria-test180": "test/images",
    "inria-train180": "train/images",
    "XView-train846": "train_images",
    "XView-val281": "val_images",
}

In [2]:
# Imports
import os
import argparse
import shutil
import mlflow
import pandas as pd
from pdb import set_trace as debug # debugging

# display tables of max 50 columns
pd.set_option('display.max_columns', 50)

from custom_modifiers import DSModifierFake
from iquaflow.datasets import DSModifier, DSWrapper
from iquaflow.experiments import ExperimentInfo, ExperimentSetup
from iquaflow.experiments.experiment_visual import ExperimentVisual
from iquaflow.experiments.task_execution import PythonScriptTaskExecution
from iquaflow.quality_metrics import RERMetrics, SNRMetrics, GaussianBlurMetrics, NoiseSharpnessMetrics, GSDMetrics, ScoreMetrics
from visual_comparison import metric_comp, plotSNE

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  """


In [3]:
# Remove previous mlflow records of previous executions of the same experiment
try: # rm_experiment
    mlflow.delete_experiment(ExperimentInfo(f"{experiment_name}").experiment_id)
    # Clean mlruns and __pycache__ folders
    shutil.rmtree("mlruns/",ignore_errors=True)
    os.makedirs("mlruns/.trash", exist_ok=True)
    shutil.rmtree(f"{data_path}/.ipynb_checkpoints",ignore_errors=True)
    [shutil.rmtree(x) for x in glob(os.path.join(os.getcwd(), "**", '__pycache__'), recursive=True)]
except:
    pass


In [4]:
dataframes = []
for ids, database_name in enumerate(list(data_paths.keys())):

    data_path = data_paths[database_name]
    images_path = os.path.join(data_paths[database_name],image_folders[database_name])
    python_ml_script_path = 'sr.py'  

    #Define name of IQF experiment
    experiment_name='eoqmr'
    experiment_name += f"_{database_name}"
    
    # set output folders
    plots_folder = "plots/"+experiment_name+"/"
    results_folder = "results/"+experiment_name+"/"
    os.makedirs(plots_folder, exist_ok=True)
    os.makedirs(results_folder, exist_ok=True)
    
    # plot SNE of existing images
    if plot_sne:
        plotSNE(database_name, data_path, (232,232), 6e4, True, savefig, plots_folder)

    #DS wrapper is the class that encapsulate a dataset
    ds_wrapper = DSWrapper(data_path=data_path)
    
    # Define and execute script (sr.py: copy image files to testing folder)
    ds_wrapper = DSWrapper(data_path=data_path)
    ds_modifiers_list = [DSModifier()] # DSModifierFake(name="base",images_dir=images_path)
    task = PythonScriptTaskExecution( model_script_path = python_ml_script_path )
    experiment = ExperimentSetup(
        experiment_name=experiment_name,
        task_instance=task,
        ref_dsw_train=ds_wrapper,
        ds_modifiers_list=ds_modifiers_list,
        ref_dsw_val=ds_wrapper,
        repetitions=1,
        extra_train_params={"trainds": [data_path], "traindsinput": [images_path], "valds": [data_path],"valdsinput": [images_path]}
    )
    # Enforce replacement of paths in experiment wrappers
    '''
    experiment.ref_dsw_train.data_input = images_path
    if experiment.ref_dsw_test is not None:
        experiment.ref_dsw_test.data_input = images_path
    if experiment.ref_dsw_val is not None:
        experiment.ref_dsw_val.data_input = images_path 
    '''

    #Execute the experiment
    experiment.execute()
    experiment_info = ExperimentInfo(experiment_name)
    # ExperimentInfo is used to retrieve all the information of the whole experiment. 
    # It contains built in operations but also it can be used to retrieve raw data for futher analysis

    print('Calculating Quality Metric Regression...'+",".join(regressor_quality_metrics)) #default configurations
    path_regressor_quality_metrics = f'./{results_folder}regressor_quality_metrics.csv'
    if use_existing_metrics and os.path.exists(path_regressor_quality_metrics):
        df = pd.read_csv(path_regressor_quality_metrics, index_col='ds_modifier')
    else:
        _ = experiment_info.apply_metric_per_run(ScoreMetrics(), ds_wrapper.json_annotations)
        _ = experiment_info.apply_metric_per_run(RERMetrics(), ds_wrapper.json_annotations)
        _ = experiment_info.apply_metric_per_run(SNRMetrics(), ds_wrapper.json_annotations)
        _ = experiment_info.apply_metric_per_run(GaussianBlurMetrics(), ds_wrapper.json_annotations)
        _ = experiment_info.apply_metric_per_run(NoiseSharpnessMetrics(), ds_wrapper.json_annotations)
        _ = experiment_info.apply_metric_per_run(GSDMetrics(), ds_wrapper.json_annotations)
        df = experiment_info.get_df(
            ds_params=["ds_modifier"],
            metrics=regressor_quality_metrics,
            dropna=False
        )
        df.to_csv(path_regressor_quality_metrics)

    # check results
    df["ds_modifier"] = database_name
    
    # Clean dataframe
    df = df[['ds_modifier']+regressor_quality_metrics]
    df = df.set_index('ds_modifier')
    df = df[~df.index.duplicated(keep='first')]
    
    print(f"writing {path_regressor_quality_metrics}")
    df.to_csv(path_regressor_quality_metrics)
    
    # append table of current dataset
    dataframes.append(df)


Directory not copied. Error: [Errno 17] File exists: './Data/DeepGlobe#base_modifier/images'
Directory not copied. Error: [Errno 17] File exists: './Data/DeepGlobe#base_modifier/images'
python sr.py --trainds ./Data/DeepGlobe#base_modifier --outputpath /tmp/tmpzoid6g6o --trainds ./Data/DeepGlobe --traindsinput ./Data/DeepGlobe/images --valds ./Data/DeepGlobe#base_modifier --valdsinput ./Data/DeepGlobe/images
****** IQF subprocess --stdout-- *********

****** IQF subprocess --stderr-- *********

Calculating Quality Metric Regression...sigma,snr,rer,sharpness,scale,score
writing ./results/eoqmr_DeepGlobe469/regressor_quality_metrics.csv
Directory not copied. Error: [Errno 17] File exists: './Data/UCMerced_LandUse#base_modifier/Images'
Directory not copied. Error: [Errno 17] File exists: './Data/UCMerced_LandUse#base_modifier/Images'
python sr.py --trainds ./Data/UCMerced_LandUse#base_modifier --outputpath /tmp/tmpc3nuz74z --trainds ./Data/UCMerced_LandUse --traindsinput ./Data/UCMerced_L

In [5]:
# concat all dataset tables
#df = pd.concat(dataframes)
results_folder = "results/"
path_all_datasets = f"{results_folder}results.csv"
print(f"writing csv: {path_all_datasets}")
df = pd.concat(dataframes,axis=0)
{df.drop(str(field),inplace=True,axis=1) for field in df if "Unnamed" in str(field)}
df.to_csv(path_all_datasets)

writing csv: results/results.csv


In [6]:
df

,sigma,snr,rer,sharpness,scale,score
ds_modifier,,,,,,
DeepGlobe469,1.017219,49.850427,0.217099,1.000000,0.418945,0.655449
UCMerced2100,1.006696,45.622857,0.231276,3.108571,0.424219,0.638085
UCMerced380,1.001913,15.000000,0.250000,1.000000,0.415430,0.848206
USGS279,1.002870,47.981183,0.206129,1.280645,0.394237,0.655312
shipsnet-ships4000,1.000000,20.000000,0.272662,3.109375,0.414844,0.787925
shipsnet-scenes7,1.008610,49.687500,0.284375,4.375000,0.390234,0.622649
inria-test10,1.002870,40.300000,0.267750,1.000000,0.415781,0.730449
inria-test180,1.026786,39.219444,0.254889,1.000000,0.445313,0.725842
inria-train180,1.009566,45.463889,0.291639,1.000000,0.353320,0.716489


In [7]:
# plot metric comparison
if plot_metrics_comp:
    metric_comp(df,regressor_quality_metrics,savefig,plots_folder)